## Step 1: load datasets
Below we generate some toy datasets using generate_toy_datasets() as defined in utils.py. User can load their own survival datasets into "datasets", which should be a list of (X, time, event) tuples, where X, time, and event are the design matrix, survival time and event vectors for a given dataset. 

In [1]:
from utils import generate_toy_datasets

n_datasets = 10 # generate 10 datasets
n_min, n_max = 100, 200 # number of samples in each dataset is a random integer between 100 and 200
n_features = 10000 # number of features is 10000
datasets = generate_toy_datasets(n_datasets, n_min, n_max, n_features)

## Step 2 (optional): feature transformation
If necessary, we can first preprocess X so that it is standardized. We provide in preprocessing.py two basic types of feature transformation functions:
- __rank_transform()__: transform features of each sample into normalized ranks
- __zscore_transform()__: transform each feature to be zero mean and unit std across samples

We can then wrap them in a FeatureTransformer object which defines the fit_transform method for our "datasets" list

In [2]:
from preprocessing import rank_transform, FeatureTransformer

feature_transformer = FeatureTransformer(rank_transform)
datasets_transformed = feature_transformer.fit_transform(datasets)

## Step 3: split into training and validation
We provide train_test_split() in utils.py to split "datasets" list in a stratified way. That is, each dataset in "datasets" is split according to test_size

In [3]:
from utils import train_test_split

datasets_train, datasets_val = train_test_split(datasets_transformed, test_size=0.2)

## Step 4 (optional): feature selection
We can additionally perform a feature selection step to reduce the number of features before model training. In feature_selection.py we provide a feature selection method based on concordance index as commonly used to characterize the feature's correlation with survival. 

Also note that our feature selection for multiple datasets is based on meta-analysis. The concordance index is calculated for each dataset and combined into a meta-score based on the size of the dataset. This is done by wrapping the score function in a SelectKBestMeta object which defines the fit and transform function. 

In [4]:
from feature_selection import concordance_index, SelectKBestMeta

topK = 1024 # select top 1024 features
feature_selector = SelectKBestMeta(concordance_index, topK)
feature_selector.fit(datasets_train)
datasets_train_new = feature_selector.transform(datasets_train)
datasets_val_new = feature_selector.transform(datasets_val)

## Step 5: user defined keras model
This is the core input required of the user. Below we provide a simple fully-connected network with 3 hidden layers. Note that there is no need to apply any activation function to the input layer. We are building a survival regression model!

In [5]:
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout
import keras.backend as K


def build_model(feature_dim):
    '''
    Define a callable keras model yourself
    model input should be a (None, feature_dim) tensor,
    model output should be a (None, 1) tensor
    '''
    x = Input(shape=(feature_dim,))
    #--------START OF USER CODE-------
    a0 = Dropout(0.3)(x)
    z1 = Dense(units=1024, activation=None)(a0)
    a1 = Activation(activation='elu')(z1)
    a1 = Dropout(0.5)(a1)
    z2 = Dense(units=1024, activation=None)(a1)
    a2 = Activation(activation='elu')(z2)
    a2 = Dropout(0.5)(a2)
    y = Dense(units=1, activation=None)(a2)
    #--------END OF USER CODE-------
    
    model = Model(inputs=x, outputs=y)
    return model

C:\Users\cuiyi\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Step 6: create a model and train
We provide a high-level class SurvivalModel to facilitate model training. In SurvivalModel.fit(), There are two modes for model training: merge or decentralized. For mode='dencentral'. each dataset will be treated as a mini-batch. For mode='merge', the datasets are merged into a single dataset and mini-batches are sampled from the merged dataset. If your datasets are very heterogenous (eg different cancers), consider mode='decentral'; otherwise, mode='merge' should be the choice.

In [6]:
from models import SurvivalModel

survival_model = SurvivalModel(build_model)
survival_model.fit(datasets_train_new, datasets_val_new, loss_func='logloss', epochs=1000, lr=0.001, mode='decentral')

C:\Users\cuiyi\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 0: loss_train=0.3781, ci_train=0.8491, loss_val=0.7635, ci_val=0.5079
Epoch 100: loss_train=0.1587, ci_train=0.9531, loss_val=0.8154, ci_val=0.5208
Epoch 200: loss_train=0.1343, ci_train=0.9628, loss_val=0.8517, ci_val=0.5194
Epoch 300: loss_train=0.1186, ci_train=0.9735, loss_val=0.8349, ci_val=0.5228
Epoch 400: loss_train=0.1055, ci_train=0.9809, loss_val=0.8361, ci_val=0.5208
Epoch 500: loss_train=0.0727, ci_train=0.9865, loss_val=0.9013, ci_val=0.5237
Epoch 600: loss_train=0.0637, ci_train=0.9888, loss_val=0.8121, ci_val=0.5534
Epoch 700: loss_train=0.0517, ci_train=0.9909, loss_val=0.8478, ci_val=0.5089
Epoch 800: loss_train=0.0505, ci_train=0.9918, loss_val=0.8127, ci_val=0.5323
Epoch 900: loss_train=0.0446, ci_train=0.9934, loss_val=0.8471, ci_val=0.5381


This model achieves an almost perfect performance on the training dataset but not so on the testing dataset. This is expected since our simulated datasets are just randomly generated and there is nothing to learn (it'll be surprising if it does learn anything useful...). You can provide your own dataset and check if it also works on testing dataset. 

## Step 7: predict on testing data

In [12]:
X_test, time_test, event_test = datasets_val_new[2]
y_pred = survival_model.predict(X_test)
cindex, _ = concordance_index(y_pred, time_test, event_test)
print(cindex)

[0.49689441]
